In [ ]:
!pip install openai

In [1]:
import openai
import os
import random
import json
from tqdm import tqdm
import tiktoken
import shutil
import time
import requests
from utils import * 
from requests.exceptions import HTTPError, RequestException
with open("chatGPT_API_KEY.txt") as f:
    API_KEY = f.read()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
openai.api_key = API_KEY
tokenizer = tiktoken.get_encoding('cl100k_base')

def getResume(res_num = None, name = None, path = "resume_corpus"):
    if res_num is None:
        res_num = random.randint(0, len(os.listdir(path)) - 1)
        
    resume = name if name else os.listdir(path)[res_num]
    file_path = os.path.join(path, resume)
    
    with open(file_path, 'r', encoding='utf-8', errors="replace") as f:
        text = f.read()
    
    return text

def getNumTokens(text, tokenizer = tokenizer):
    return len(tokenizer.encode(text))

In [2]:
with open("resumes_groupby_tokens/resumes_400_600.txt") as f:
    resumes = f.read().split("\n")
    
message_list = [[{'role': "system", "content": SYSINS},
                    {"role": "user", "content": INSTRUCTION + getCleanResume(res)}]
                for res in resumes[:10]]


In [5]:
print(message_list[0][1]['content'])

Please parse the resume into the following JSON format: 
{
    "Basic_Info": {
        "Current_Title": "",
        "Current_Company": "",
        "Location": "",
        "Bio": ""
    },
    "Experience": [
        {
            "Job_Title": "",
            "Company": "",
            "Location": "",
            "Start_Date": "",
            "End_Date": "",
            "Responsibilities": ""
        }
    ],
    "Education": [
        {
            "Degree": "",
            "Field": "",
            "Institution": "",
            "Location": "",
            "Graduation_Date": ""
        }
    ],
    "Projects": [
        {
            "Project_Title": "",
            "Description": "",
            "Date": ""
        }
    ],
    "Skills": [],
    "Technical_Skills": [],
    "Links": [],
    "Certifications": [
        {
            "Certification_Title": "",
            "Issuing_Organization": "",
            "Date_Issued": ""
        }
    ],
    "Awards": [
        {
            "Awar

In [3]:
GPT_35_16K = "gpt-3.5-turbo-16k"
GPT_35 = "gpt-3.5-turbo"
GPT_4 = "gpt-4"

            
output_format_skeleton = """
{
    "Basic_Info": {
        "Current_Title": "",
        "Current_Company": "",
        "Location": "",
        "Bio": ""
    },
    "Experience": [
        {
            "Job_Title": "",
            "Company": "",
            "Location": "",
            "Start_Date": "",
            "End_Date": "",
            "Responsibilities": ""
        }
    ],
    "Education": [
        {
            "Degree": "",
            "Field": "",
            "Institution": "",
            "Location": "",
            "Graduation_Date": ""
        }
    ],
    "Projects": [
        {
            "Project_Title": "",
            "Description": "",
            "Date": ""
        }
    ],
    "Skills": [],
    "Technical_Skills": [],
    "Links": [],
    "Certifications": [
        {
            "Certification_Title": "",
            "Issuing_Organization": "",
            "Date_Issued": ""
        }
    ],
    "Awards": [
        {
            "Award_Title": "",
            "Issuing_Organization": ""
        }
    ],
    "Publications": [
        {
            "Publication_Title": "",
            "Date": ""
        }
    ]
}"""
            

In [4]:
path = "resume_corpus"
#task = "turn the following resumes into JSON format with their file names as their keys not including the file extension. Parse the ENTIRE resume, and do not omit any part due to length! Do not add any extra sections!"
instruction = f"Please parse the resume into the following JSON format: \n{output_format_skeleton}"
resume = getResume(name = "03061.txt")
#print(resume)

In [5]:
print(instruction + resume)

Please parse the resume into the following JSON format: 

{
    "Basic_Info": {
        "Current_Title": "",
        "Current_Company": "",
        "Location": "",
        "Bio": ""
    },
    "Experience": [
        {
            "Job_Title": "",
            "Company": "",
            "Location": "",
            "Start_Date": "",
            "End_Date": "",
            "Responsibilities": ""
        }
    ],
    "Education": [
        {
            "Degree": "",
            "Field": "",
            "Institution": "",
            "Location": "",
            "Graduation_Date": ""
        }
    ],
    "Projects": [
        {
            "Project_Title": "",
            "Description": "",
            "Date": ""
        }
    ],
    "Skills": [],
    "Technical_Skills": [],
    "Links": [],
    "Certifications": [
        {
            "Certification_Title": "",
            "Issuing_Organization": "",
            "Date_Issued": ""
        }
    ],
    "Awards": [
        {
            "Awa

In [6]:
a = list(range(26))
for i in range(0, 26, 10):
    print(a[i:i+10])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[20, 21, 22, 23, 24, 25]


In [22]:
# with open("parsed_resume.json") as f:
#     parsed_resume = json.load(f)

# avai_res = [x for x in os.listdir("resume_corpus") if x not in parsed_resume.keys()]
# res_to_use = random.sample(avai_res, 10)
res_to_use = list(temp.keys())
sample_parse_res = {}

In [ ]:
for res in tqdm(res_to_use):
    resume = getResume(name = res)
    response = openai.ChatCompletion.create(
        model = GPT_35_16K,
        messages = [
            {'role': "system", "content": "You are a detail oriented resume parser who does not miss any information."},
            {"role": "user", "content": instruction + resume}
            ],
        temperature = 0,
        presence_penalty = -0.8
    )
    
    sample_parse_res[res] = eval(response['choices'][0]["message"]["content"])

In [26]:
with open("sample_Chatgpt_parse_res.json", "w") as f:
    json.dump(sample_parse_res, f, indent=4)

In [27]:
with open("llama_parsed_resume.json") as f:
    temp = json.load(f)
temp = {k: v for k, v in temp.items() if k in sample_parse_res.keys()}
with open("sample_Llama2_parse_res.json", "w") as f:
    json.dump(temp, f, indent=4)
# with open("sample_parse_res.json", "w") as f:
#     json.dump(sample_parse_res, f, indent=4)
temp.keys()

dict_keys(['21770.txt', '08995.txt', '03249.txt', '03114.txt', '16860.txt', '02107.txt', '06375.txt', '01690.txt', '19202.txt', '25012.txt'])

In [29]:
gpt35_response_dict = {}
gpt35_resume_dict = {}
jj = 0
path = "resume_corpus"



In [171]:
# 'resume_03061', 'resume_25918', 'resume_18327', 'resume_00568',

def joinResumes(resume_list, path = path):
    joint_resume = ""
    
    for resume in resume_list:
        with open(os.path.join(path, resume), 'r', encoding='utf-8', errors="replace") as f:
            text = f.read()
        joint_resume += f"Resume {resume}: \n{text} \n\n"
        
    return joint_resume
    
    
def processBatches(model, total_num_resume, resume_dict, response_dict, tokens_limit, task=task, output_format=output_format, path=path, start_from_index=0, already_processed = None):
    jj = start_from_index
    encoding = tiktoken.encoding_for_model(GPT_4)

    # Create a new directory for long resumes if it does not exist
    if not os.path.exists(os.path.join(path, 'resume_long')):
        os.makedirs(os.path.join(path, 'resume_long'))
    
    task_output_format_tokens = len(encoding.encode(task + output_format))
    resume_tokens_limit = tokens_limit - task_output_format_tokens
    
    last_saved_time = time.time() 
    
    with open('processed_resumes.txt', 'a') as processed_resumes_file, open('failed_resumes.txt', 'a') as failed_resumes_file:
        while jj < total_num_resume:
            joint_resume = ""
            current_batch_resume_names = []  # to keep track of resume names in current batch
            print(f"processing resumes starting from index {jj}")

            while True:
                resume = os.listdir(path)[jj]

                # Skip the resume if it has already been processed
                if already_processed is not None and resume in already_processed:
                    jj += 1
                    total_num_resume += 1
                    print(f"Resume {resume} has already been processed, skipping it.")
                    continue

                file_path = os.path.join(path, resume)
                with open(file_path, 'r', encoding='utf-8', errors="replace") as f:
                    text = f.read()

                # Check if the single resume's token count is more than the token limit
                single_resume_tokens = len(encoding.encode(f" Resume {resume}: \n {text} \n\n"))
                if single_resume_tokens > resume_tokens_limit:
                    print(f"Single resume's tokens exceeded the limit, moving {resume} to resume_long folder.")
                    shutil.move(file_path, os.path.join(path, 'resume_long', resume))
                    jj += 1
                    continue

                joint_resume_temp = joint_resume + f" Resume {resume}: \n {text} \n\n"
                tokens = len(encoding.encode(joint_resume_temp))

                if tokens > resume_tokens_limit:
                    print(f"Tokens limit exceeded with resume #{jj} {resume}, it will be processed in the next batch.")
                    break
                else:
                    joint_resume = joint_resume_temp
                    current_batch_resume_names.append(resume)  # add the resume name to the current batch list
                    jj += 1

                if jj >= total_num_resume:
                    break

            job = task + output_format + joint_resume

            print(f"Total JOB tokens: {len(encoding.encode(job))}\n")
            
            try: 
                response = openai.ChatCompletion.create(
                            model = model,
                            messages = [
                                {'role': "system", "content": "You are a detail oriented resume parser who does not miss any information."},
                                {"role": "user", "content": job},
                            ],
                            temperature = 0,
                            presence_penalty = -0.8
                            )
            except HTTPError as e:
                if response.status_code == 502:
                    print("HTTPError 502: Bad Gateway. Trying again in 10 seconds.")
                    time.sleep(10)
                else:
                    print("HTTPError: ", str(e))
                
            except RequestException as e:
                print(f"There was a problem with the request: {str(e)}")
            
            if response.choices[0].finish_reason != 'stop':
                resume_tokens_limit *= 0.9
                jj -= len(current_batch_resume_names)
                print("WARNING: response did not finish. Reducing token limit by factor of 0.1\nCurrent token limit: ", tokens_limit)
                continue
            
            extracted_resume = response['choices'][0]["message"]["content"] + "\n\n"
            response_dict[f"{jj-len(current_batch_resume_names)}_{jj-1}"] = response
            try:
                resume_dict.update(eval(extracted_resume))
                processed_resumes_file.write(resume + '\n')

            except Exception as e:
                print(f"Failed to parse the resume due to {str(e)}. Recording the failed resumes.")
                failed_resumes_file.write('\n'.join(current_batch_resume_names) + '\n')  # write the failed resume names
            
            print(f"Finished processing resumes up to index {jj}. \t Total tokens: {response['usage'].total_tokens}\n")
            
            current_time = time.time()
            if current_time - last_saved_time > 8 * 60:  # 8 minutes have passed
                with open('parsed_resume.json', 'w') as json_file:
                    json.dump(resume_dict, json_file, indent = 2)
                print("saved the resume dict as json file")
                last_saved_time = current_time  
                
        end_index = jj
    
    return resume_dict, response_dict, end_index

    
# Usage started at $1.72
# Total started at $107.65
    


In [201]:
already_processed = [x.split('_')[1]+'.txt' if "_" in x else x + ".txt" for x in list(gpt35_resume_dict.keys())]
len(already_processed)

4435

-3.42671009771987

In [199]:
path = "resume_corpus"
task = "turn the following resumes into JSON format with their file names as their keys not including the file extension. Parse the ENTIRE resume, and do not omit any part due to length! Do not add any extra sections!"
already_processed = [x.split('_')[1]+'.txt' if "_" in x else x + ".txt" for x in list(gpt35_resume_dict.keys())]

processBatches(model = GPT_35_16K, 
               total_num_resume = 4802, 
               resume_dict = gpt35_resume_dict, 
               response_dict = gpt35_response_dict, 
               tokens_limit = 16384 // 2.8,
               output_format = output_format_skeleton,
               start_from_index = 4359,
               already_processed = already_processed);

processing resumes starting from index 4359
Tokens limit exceeded with resume #4364 18527.txt, it will be processed in the next batch.
Total JOB tokens: 5183

Finished processing resumes up to index 4364. 	 Total tokens: 10840

processing resumes starting from index 4364
Tokens limit exceeded with resume #4366 01076.txt, it will be processed in the next batch.
Total JOB tokens: 3433

Finished processing resumes up to index 4366. 	 Total tokens: 7271

processing resumes starting from index 4366
Tokens limit exceeded with resume #4369 11265.txt, it will be processed in the next batch.
Total JOB tokens: 5751

Finished processing resumes up to index 4369. 	 Total tokens: 10892

processing resumes starting from index 4369
Tokens limit exceeded with resume #4371 08720.txt, it will be processed in the next batch.
Total JOB tokens: 5493

Finished processing resumes up to index 4371. 	 Total tokens: 11462

processing resumes starting from index 4371
Tokens limit exceeded with resume #4373 18533

ServiceUnavailableError: The server is overloaded or not ready yet.

In [ ]:
with open("parsed_resume.json", "w") as f:
    json.dump(gpt35_resume_dict, f, indent = 2)

In [166]:
with open("parsed_resume_backup.json", "w") as f:
    json.dump(gpt35_resume_dict, f, indent = 2)



In [192]:
with open("parsed_resume.json", "r") as f:
    print(len(json.load(f)))

2687
